# Classification of Data: PointNet++

The first step in this process is to classify all unclassified data in the LiDAR dataset. This will ensure proper building skeletons prior to densification. 

We will be training a PointNet++ model to do this; The data will be trained on the already classified data. 

In [1]:
# Data preprocessing
import pylas
import os
import tqdm
import src.utils.data_helper as data_helper
import config
import pickle

config = config.get_config()

In [2]:

for filename in tqdm.tqdm(os.listdir(config.lidar_directory)):
    if filename.endswith(".las"):
        las = pylas.read(os.path.join(config.lidar_directory, filename))
        classified_las, unclassified_las = data_helper.split_las(las)
        data_helper.convert_to_pkl(classified_las, os.path.join(config.pkl_classified_directory, filename[:-4] + ".pkl"))
        data_helper.convert_to_pkl(unclassified_las, os.path.join(config.pkl_unclassified_directory, filename[:-4] + ".pkl"))

# merge the pkl files into a single file (for each classified and unclassified)


100%|██████████| 27/27 [00:15<00:00,  1.72it/s]


In [3]:
data_helper.merge_pkl_files(config.pkl_classified_directory, os.path.join(config.pkl_classified_directory, "classified_merged.pkl"))
data_helper.merge_pkl_files(config.pkl_unclassified_directory, os.path.join(config.pkl_unclassified_directory, "unclassified_merged.pkl"))

Unclassified
Ground
Low Vegetation
Medium Vegetation
High Vegetation
Building
Low Point (Noise)
Reserved
Water
Rail
Road Surface
Reserved
Wire - Guard (Shield)
Wire - Conductor (Phase)
Transmission Tower
Wire-Structure Connector (Insulator)
 Bridge Deck
High Noise

# Classification Training

In [4]:
NUM_CLASSES = 17  # not taking into account unclassified. Not all of these classes are used; TODO check if it's okay that the latter half never have any points
points_dir = os.path.join(config.pkl_classified_directory, "classified_merged_points.pkl")
labels_dir = os.path.join(config.pkl_classified_directory, "classified_merged_labels.pkl")

with open(points_dir, "rb") as f:
    points = pickle.load(f)
    
with open(labels_dir, "rb") as f:
    labels = pickle.load(f)
    
